<a href="https://colab.research.google.com/github/JeehwanLim/202002_NLP_FIN/blob/main/%EA%B8%B0%EB%A7%90%EA%B3%BC%EC%A0%9C_%ED%95%9C%EA%B8%802.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 사전 설정

In [1]:
# HuggingFace transformers 설치 및 NSMC 데이터셋 다운로드
!pip install transformers

import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

     |████████████████████████████████| 1.5MB 4.3MB/s 
     |████████████████████████████████| 890kB 11.0MB/s 
     |████████████████████████████████| 2.9MB 23.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=e7d0cfca9cff45d574df48b32c0165d0faca4830311f0d9e7879dd9fbb12b01d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [3]:
##GPU 사용 시
device = torch.device("cuda:0")

In [4]:
!git clone https://github.com/JeehwanLim/202002_NLP_FIN.git

Cloning into '202002_NLP_FIN'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 9), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), done.


In [5]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 9.04 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


# 데이터 전처리

In [6]:
class NSMCDataset(Dataset):
  
  def __init__(self, csv_file):
    # 일부 값중에 NaN이 있음...
    self.dataset = pd.read_csv(csv_file, sep='\t').dropna(axis=0) 
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")

    print(self.dataset.describe())
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 1:3].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [7]:
class NSMCDataset_test(Dataset):
  
  def __init__(self, csv_file):
    # 일부 값중에 NaN이 있음...
    self.dataset = pd.read_csv(csv_file, sep=',', encoding='cp949')
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")

    print(self.dataset.describe())
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 1:2].values
    text = row[0]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask

In [8]:
train_dataset = NSMCDataset("nsmc/ratings_train.txt")
validation_dataset = NSMCDataset("nsmc/ratings_test.txt")
test_dataset = NSMCDataset_test("202002_NLP_FIN/ko_data.csv")


                 id         label
count  1.499950e+05  149995.00000
mean   6.743648e+06       0.49885
std    2.919001e+06       0.50000
min    3.300000e+01       0.00000
25%    4.767467e+06       0.00000
50%    7.526885e+06       0.00000
75%    9.249448e+06       1.00000
max    1.027815e+07       1.00000
                 id         label
count  4.999700e+04  49997.000000
mean   6.728781e+06      0.503450
std    2.936634e+06      0.499993
min    6.010000e+02      0.000000
25%    4.736525e+06      0.000000
50%    7.530895e+06      1.000000
75%    9.246512e+06      1.000000
max    1.027809e+07      1.000000
                 Id
count  11187.000000
mean    5593.000000
std     3229.553065
min        0.000000
25%     2796.500000
50%     5593.000000
75%     8389.500000
max    11186.000000


# 모델 생성 및 학습

In [9]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator").to(device)

Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized

In [10]:
# model.load_state_dict(torch.load("koelectra-small_1.pt"))

In [11]:
epochs = 5
batch_size = 128
learning_rate = 2e-5

In [12]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=16)
test_loader = DataLoader(test_dataset)

In [13]:
losses = []
accuracies = []

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Batch Loss: 66.99607008695602 Accuracy: tensor(0.6032, device='cuda:0')
Batch Loss: 117.31319853663445 Accuracy: tensor(0.6962, device='cuda:0')
Batch Loss: 160.27613127231598 Accuracy: tensor(0.7360, device='cuda:0')
Batch Loss: 199.80442470312119 Accuracy: tensor(0.7596, device='cuda:0')
Batch Loss: 237.9126960337162 Accuracy: tensor(0.7749, device='cuda:0')
Batch Loss: 274.2532186806202 Accuracy: tensor(0.7865, device='cuda:0')
Batch Loss: 309.8949036002159 Accuracy: tensor(0.7951, device='cuda:0')
Batch Loss: 343.5881712734699 Accuracy: tensor(0.8026, device='cuda:0')
Batch Loss: 377.91602343320847 Accuracy: tensor(0.8079, device='cuda:0')
Batch Loss: 411.3462366461754 Accuracy: tensor(0.8127, device='cuda:0')
Batch Loss: 445.3852984905243 Accuracy: tensor(0.8164, device='cuda:0')

Train Loss: 469.51170724630356 Accuracy: tensor(0.8188, device='cuda:0')


Batch Loss: 31.283232659101486 Accuracy: tensor(0.8687, device='cuda:0')
Batch Loss: 61.487705051898956 Accuracy: tensor(0.8708, device='cuda:0')
Batch Loss: 91.88641157746315 Accuracy: tensor(0.8707, device='cuda:0')
Batch Loss: 122.63551603257656 Accuracy: tensor(0.8702, device='cuda:0')
Batch Loss: 153.02530105412006 Accuracy: tensor(0.8706, device='cuda:0')
Batch Loss: 184.04138532280922 Accuracy: tensor(0.8701, device='cuda:0')
Batch Loss: 214.0258348286152 Accuracy: tensor(0.8702, device='cuda:0')
Batch Loss: 243.984290137887 Accuracy: tensor(0.8706, device='cuda:0')
Batch Loss: 272.4339121133089 Accuracy: tensor(0.8716, device='cuda:0')
Batch Loss: 302.85093796253204 Accuracy: tensor(0.8716, device='cuda:0')
Batch Loss: 332.1988161057234 Accuracy: tensor(0.8718, device='cuda:0')

Train Loss: 353.12451937794685 Accuracy: tensor(0.8721, device='cuda:0')


Batch Loss: 27.463401719927788 Accuracy: tensor(0.8852, device='cuda:0')
Batch Loss: 55.495528772473335 Accuracy: tensor(0.8829, device='cuda:0')
Batch Loss: 83.13691766560078 Accuracy: tensor(0.8823, device='cuda:0')
Batch Loss: 110.48877443373203 Accuracy: tensor(0.8830, device='cuda:0')
Batch Loss: 138.07832400500774 Accuracy: tensor(0.8831, device='cuda:0')
Batch Loss: 165.6905729174614 Accuracy: tensor(0.8834, device='cuda:0')
Batch Loss: 192.1084317713976 Accuracy: tensor(0.8841, device='cuda:0')
Batch Loss: 218.3788393586874 Accuracy: tensor(0.8849, device='cuda:0')
Batch Loss: 245.13061283528805 Accuracy: tensor(0.8851, device='cuda:0')
Batch Loss: 272.09209029376507 Accuracy: tensor(0.8852, device='cuda:0')
Batch Loss: 299.56809552013874 Accuracy: tensor(0.8851, device='cuda:0')

Train Loss: 319.11115603893995 Accuracy: tensor(0.8851, device='cuda:0')


Batch Loss: 25.479200586676598 Accuracy: tensor(0.8946, device='cuda:0')
Batch Loss: 51.16823038458824 Accuracy: tensor(0.8941, device='cuda:0')
Batch Loss: 77.26781342923641 Accuracy: tensor(0.8930, device='cuda:0')
Batch Loss: 102.45947507023811 Accuracy: tensor(0.8931, device='cuda:0')
Batch Loss: 128.2530602067709 Accuracy: tensor(0.8927, device='cuda:0')
Batch Loss: 152.85011534392834 Accuracy: tensor(0.8936, device='cuda:0')
Batch Loss: 177.6731902360916 Accuracy: tensor(0.8939, device='cuda:0')
Batch Loss: 202.54824410378933 Accuracy: tensor(0.8943, device='cuda:0')
Batch Loss: 228.41424663364887 Accuracy: tensor(0.8938, device='cuda:0')
Batch Loss: 253.36286471784115 Accuracy: tensor(0.8941, device='cuda:0')
Batch Loss: 277.6612210869789 Accuracy: tensor(0.8946, device='cuda:0')

Train Loss: 295.6771279126406 Accuracy: tensor(0.8947, device='cuda:0')


Batch Loss: 24.357294663786888 Accuracy: tensor(0.9030, device='cuda:0')
Batch Loss: 48.04437208175659 Accuracy: tensor(0.9023, device='cuda:0')
Batch Loss: 71.62389840185642 Accuracy: tensor(0.9024, device='cuda:0')
Batch Loss: 95.12859083712101 Accuracy: tensor(0.9025, device='cuda:0')
Batch Loss: 118.19309267401695 Accuracy: tensor(0.9029, device='cuda:0')
Batch Loss: 140.85855663567781 Accuracy: tensor(0.9038, device='cuda:0')
Batch Loss: 164.1669385433197 Accuracy: tensor(0.9037, device='cuda:0')
Batch Loss: 187.97260773181915 Accuracy: tensor(0.9035, device='cuda:0')
Batch Loss: 212.69684816896915 Accuracy: tensor(0.9028, device='cuda:0')
Batch Loss: 236.43187221884727 Accuracy: tensor(0.9026, device='cuda:0')
Batch Loss: 259.0873518586159 Accuracy: tensor(0.9030, device='cuda:0')

Train Loss: 275.79048189520836 Accuracy: tensor(0.9032, device='cuda:0')


In [14]:
losses, accuracies

([469.51170724630356,
  353.12451937794685,
  319.11115603893995,
  295.6771279126406,
  275.79048189520836],
 [tensor(0.8188, device='cuda:0'),
  tensor(0.8721, device='cuda:0'),
  tensor(0.8851, device='cuda:0'),
  tensor(0.8947, device='cuda:0'),
  tensor(0.9032, device='cuda:0')])

# Validation set 정확도 확인

In [15]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(validation_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



Accuracy: tensor(0.8894, device='cuda:0')


In [20]:
# 모델 저장하기
torch.save(model.state_dict(), "koelectra-small_1.pt")

# 제출 파일 생성

In [17]:
model.eval()

num = 0
f = open("ko_pred.csv", "w")
f.write("Id" + ',' + "Predicted" + '\n')

for input_ids_batch, attention_masks_batch in tqdm(test_loader):
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  f.write(str(num) + ',' + str(predicted[0].cpu().numpy()) + '\n')

  num += 1

f.close()

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
